In [1]:
from lib import module
import time
import os, itertools
import pickle
from sklearn.metrics import f1_score, accuracy_score
import statistics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, RandomizedSearchCV, GridSearchCV
import pandas as pd
import numpy as np
from torch_geometric.data import Data
import os
import torch
import argparse
from tqdm import tqdm
import errno
import warnings

base_path = ''
feature_networks_integration = ['clinical', 'cna', 'exp','coe','met','mut'] # datatypes to concatanate node features from
learning_rate = 0.001
hid_size = 128

# optimize for optional feature selection of node features
feature_selection_per_network = [False, False, False,False, False, False]
top_features_per_network = [50, 50, 50,50,50,50]
optional_feat_selection = False
boruta_runs = 100
boruta_top_features = 50


# fixed
max_epochs = 500
min_epochs = 200
patience = 30

# fixed to get the same results from the tool each time
random_state = 404

# SUPREME run
print('SUPREME is setting up!')

dataset_name = 'full_data'

path = base_path + "data/" + dataset_name
if not os.path.exists(path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), path)
        
device = torch.device('cuda:1')

# node_networks = ['exp','coe','met'] # datatypes to use networks from


SUPREME is setting up!


In [2]:
criterion = torch.nn.CrossEntropyLoss()
data_path_node =  base_path + 'data/' + dataset_name +'/'
run_name = 'SUPREME_'+  dataset_name + '_results'
save_path = base_path + run_name + '/'

if not os.path.exists(base_path + run_name):
    os.makedirs(base_path + run_name + '/')

file = base_path + 'data/' + dataset_name +'/labels.pkl'
with open(file, 'rb') as f:
    labels = pickle.load(f)

file = base_path + 'data/' + dataset_name + '/mask_values.pkl'
if os.path.exists(file):
    with open(file, 'rb') as f:
        train_valid_idx, test_idx = pickle.load(f)
    print('use pre-defined split')
else:
    train_valid_idx, test_idx= train_test_split(np.arange(len(labels)), test_size=0.20, shuffle=True, stratify=labels)
    print('use random split')
start = time.time()

    
x_lists = []
for netw in feature_networks_integration:
    file = base_path + 'data/' + dataset_name +'/'+ netw +'.pkl'
    with open(file, 'rb') as f:
        feat = pickle.load(f)
        values = feat.values
        x_lists.append(values)
        
new_x = np.concatenate(x_lists,-1)
scaler = StandardScaler()
scaled_new_x = scaler.fit_transform(new_x)
new_x = torch.tensor(scaled_new_x,dtype=torch.float32).to(device)



train_idx,valid_idx = train_test_split(train_valid_idx,test_size=0.2)
train_mask = np.array([i in set(train_idx) for i in range(new_x.shape[0])])
valid_mask = np.array([i in set(valid_idx) for i in range(new_x.shape[0])])
test_mask = np.array([i in set(test_idx) for i in range(new_x.shape[0])])
y_test = pd.DataFrame(labels[test_mask].cpu().numpy()).values.ravel()

in_size = new_x.shape[1]
out_size = torch.unique(labels).shape[0]
new_x = new_x.cpu().numpy()
labels = labels.cpu().numpy()

use pre-defined split


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from autosklearn.classification import AutoSklearnClassifier

X_train = new_x[train_mask]
X_val = new_x[valid_mask]
X_test = new_x[test_mask]
X_train_val = new_x[train_valid_idx]

y_train = labels[train_mask]
y_val = labels[valid_mask]
y_train_val = labels[train_valid_idx]
y_test = labels[test_mask]


# Initialize AutoSklearn classifier
automl = AutoSklearnClassifier(
    time_left_for_this_task=120, # You can change this as per your requirements
    per_run_time_limit=30,       # You can change this as per your requirements
    n_jobs=8,                    # Use all available cores
    memory_limit=6144
)

# Fit on training data
automl.fit(X_train_val, y_train_val, dataset_name='classification_dataset')

# Validation 
y_val_pred = automl.predict(X_val)
val_accuracy = np.mean(y_val_pred == y_val)
print(f"Validation Accuracy: {val_accuracy:.2f}")

# Test
y_test_pred = automl.predict(X_test)
test_accuracy = np.mean(y_test_pred == y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Get the final ensemble constructed by auto-sklearn
print(automl.show_models())


[WARNING] [2023-10-26 22:52:34,176:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-10-26 22:52:35,168:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-10-26 22:52:35,588:Client-EnsembleBuilder] No runs were available to build an ensemble from
Validation Accuracy: 0.92
Test Accuracy: 0.85
{2: {'model_id': 2, 'rank': 1, 'cost': 0.2417582417582418, 'ensemble_weight': 0.02, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f8cd895b5e0>, 'balancing': Balancing(random_state=1), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f8ad70ecaf0>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f8ad70ecb80>, 'sklearn_classifier': RandomForestClassifier(max_features=57, n_estimators=512, n_jobs=1,
                       random_state=1, warm_start=True)